In [1]:
from preprocess import *
from model import *

In [2]:
df = load_temperature_df()

In [3]:
df = one_hot_encoder(df)

In [4]:
df = encode_timestamps(df)

In [5]:
df = apply_feature_engineering(df)

In [6]:
#df = df.fillna(0)

In [7]:
# val_pct is the percent of training data to use
# %training data = 1 - test_pct
train, val, test = data_split(df, val_pct=0.3, test_pct=0.2)

In [8]:
train, min_max_og_vals = normalize_training_set(train)

In [9]:
val = normalize_testing_set(val, min_max_og_vals)
test = normalize_testing_set(test, min_max_og_vals)

In [10]:
target_cols = ['TBD910OGValue', 'TBD911OGValue', 'TBD912OGValue']
feature_cols = train.columns
num_days_input = 30
C = Config(num_days_input, feature_cols, target_cols)

X_train, y_train, train_mask = create_sequences(train, C)
X_val, y_val, val_mask = create_sequences(val, C)
X_test, y_test, test_mask = create_sequences(test, C)

print("(num_sequences, num_days, num_feautures)")
print(f"X_train shape (num_sequences, num_days, num_feautures): {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

(num_sequences, num_days, num_feautures)
X_train shape (num_sequences, num_days, num_feautures): (872, 30, 38), y_train shape: (872, 3)
X_test shape: (293, 30, 38), y_test shape: (293, 3)


In [11]:
train_mask.shape, y_train.shape

((872, 3), (872, 3))

In [12]:
batch_size = 32
train_dataset = create_tf_dataset(X_train, y_train, train_mask, batch_size)
val_dataset = create_tf_dataset(X_val, y_val, val_mask, batch_size)
test_dataset = create_tf_dataset(X_test, y_test, test_mask, batch_size)

2024-12-13 17:53:19.890892: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-12-13 17:53:19.890919: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-13 17:53:19.890926: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-13 17:53:19.891230: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-13 17:53:19.891254: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 30, 38), dtype=tf.float64, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 3), dtype=tf.int64, name=None))>

In [14]:

M = CNN_Model(C)

In [15]:
M.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 16, 64)         │        36,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 10, 32)         │        10,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 8, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,163 (688.77 KB)

 Trainable params: 88,163 (688.77 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
MP = ModelParams(
    model=M,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss_fn=masked_loss,
    train_mse=tf.keras.metrics.Mean("train_mse"),
    val_mse=tf.keras.metrics.Mean("val_mse"),
    batch_size=32,
    epochs=10,
    train_data=train_dataset,
    val_data=val_dataset,
    test_data=test_dataset
)

: 

In [ ]:
with tf.device('/GPU:0'):
    train_model(MP)

2024-12-13 17:53:20.501374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
